<a href="https://colab.research.google.com/github/nptan2005/spark401_colab/blob/main/notebooks/spark_level_2_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install -y openjdk-17-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jre session-migration x11-utils
Suggested packages:
  gvfs libxt-doc openjdk-17-demo openjdk-17-source visualvm mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jdk openjdk-17-jr

In [3]:
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz

--2025-12-30 02:21:25--  https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548955321 (524M) [application/x-gzip]
Saving to: ‘spark-4.0.1-bin-hadoop3.tgz’

spark-4.0.1-bin-had 100%[===================>] 523.52M  15.0MB/s    in 32s     

2025-12-30 02:21:58 (16.3 MB/s) - ‘spark-4.0.1-bin-hadoop3.tgz’ saved [548955321/548955321]



In [4]:
# ===============================
# Spark 4.0.1 Setup (REQUIRED)
# ===============================
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark401-Training") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("Spark version:", spark.version)

Spark version: 4.0.1


# 🚀 Spark Level 2 – Bài 2: Filter, Select, WithColumn & Execution Plan

## 🎯 Mục tiêu bài này

**Sau bài này bạn sẽ:**
>*	Hiểu Transformation vs Action
>*	Biết Spark có chạy ngay hay không
>*	Bắt đầu chạm vào shuffle (rất quan trọng)

---

## 🧪 Bài toán

**Dữ liệu giao dịch:**

|order_id|customer_id|amount|country|
|--------|-----------|------|-------|
|1|C001|120|VN|
|2|C002|80|VN|
|3|C003|200|SG|
|4|C001|50|VN|




### 📌 Bước 1 – Tạo DataFrame:

In [5]:
from pyspark.sql import functions as F

data = [
    (1, "C001", 120, "VN"),
    (2, "C002", 80, "VN"),
    (3, "C003", 200, "SG"),
    (4, "C001", 50, "VN"),
]

columns = ["order_id", "customer_id", "amount", "country"]

df = spark.createDataFrame(data, columns)
df.show()

+--------+-----------+------+-------+
|order_id|customer_id|amount|country|
+--------+-----------+------+-------+
|       1|       C001|   120|     VN|
|       2|       C002|    80|     VN|
|       3|       C003|   200|     SG|
|       4|       C001|    50|     VN|
+--------+-----------+------+-------+



### 📌 Bước 2 – Transformation (KHÔNG chạy ngay)

```python
df_vn = df.filter(df.country == "VN")
df_vn_high = df_vn.filter(df_vn.amount > 100)
```

>❓ Spark đã chạy chưa?

>👉 CHƯA


In [6]:
df_vn = df.filter(df.country == "VN")
df_vn_high = df_vn.filter(df_vn.amount > 100)

### 📌 Bước 3 – Action (Spark bắt đầu chạy)

```python
df_vn_high.show()
```

#### 🧠 Nguyên lý quan trọng

|Loại|Ví dụ|
|----|-----|
|Transformation|filter, select, withColumn|
|Action|show, count, collect|

> 👉 Spark lazy execution


In [7]:
df_vn_high.show()

+--------+-----------+------+-------+
|order_id|customer_id|amount|country|
+--------+-----------+------+-------+
|       1|       C001|   120|     VN|
+--------+-----------+------+-------+



### 📌 Bước 4 – Thêm cột mới

In [7]:
df2 = df.withColumn(
    "amount_category",
    F.when(df.amount >= 100, "HIGH").otherwise("LOW")
)

df2.show()

+--------+-----------+------+-------+---------------+
|order_id|customer_id|amount|country|amount_category|
+--------+-----------+------+-------+---------------+
|       1|       C001|   120|     VN|           HIGH|
|       2|       C002|    80|     VN|            LOW|
|       3|       C003|   200|     SG|           HIGH|
|       4|       C001|    50|     VN|            LOW|
+--------+-----------+------+-------+---------------+



### 📌 Bước 5 – Xem Execution Plan:

```python
df2.explain(True)
```

Result:

```code
== Physical Plan ==
*(1) Project
+- *(1) Scan ExistingRDD
```

> 👉 Chưa có shuffle → nhẹ

In [9]:
df2.explain(True)

== Parsed Logical Plan ==
'Project [unresolvedstarwithcolumns(amount_category, CASE WHEN '`>=`(amount#2L, 100) THEN HIGH ELSE LOW END, None)]
+- LogicalRDD [order_id#0L, customer_id#1, amount#2L, country#3], false

== Analyzed Logical Plan ==
order_id: bigint, customer_id: string, amount: bigint, country: string, amount_category: string
Project [order_id#0L, customer_id#1, amount#2L, country#3, CASE WHEN (amount#2L >= cast(100 as bigint)) THEN HIGH ELSE LOW END AS amount_category#30]
+- LogicalRDD [order_id#0L, customer_id#1, amount#2L, country#3], false

== Optimized Logical Plan ==
Project [order_id#0L, customer_id#1, amount#2L, country#3, CASE WHEN (amount#2L >= 100) THEN HIGH ELSE LOW END AS amount_category#30]
+- LogicalRDD [order_id#0L, customer_id#1, amount#2L, country#3], false

== Physical Plan ==
*(1) Project [order_id#0L, customer_id#1, amount#2L, country#3, CASE WHEN (amount#2L >= 100) THEN HIGH ELSE LOW END AS amount_category#30]
+- *(1) Scan ExistingRDD[order_id#0L,custom

# 🚀 Spark Level 2 – Bài 3: groupBy, agg & SHUFFLE (cốt lõi Spark)

> **Đây là bài QUAN TRỌNG NHẤT trước khi bạn làm Spark thật sự trong CDP / Dataproc / EMR**


## 🎯 Mục tiêu bài này

**Sau bài này bạn sẽ:**
>*	Hiểu shuffle là gì (đúng bản chất)
>*	Biết vì sao groupBy rất đắt
>*	Đọc được execution plan có shuffle
>*	Biết khi nào Spark scale / khi nào chết

## 1️⃣ Bài toán

Dữ liệu order (như trước):

In [8]:
from pyspark.sql import functions as F

data = [
    (1, "C001", 120, "VN"),
    (2, "C002", 80, "VN"),
    (3, "C003", 200, "SG"),
    (4, "C001", 50, "VN"),
    (5, "C002", 70, "SG"),
]

columns = ["order_id", "customer_id", "amount", "country"]
df = spark.createDataFrame(data, columns)
df.show()

+--------+-----------+------+-------+
|order_id|customer_id|amount|country|
+--------+-----------+------+-------+
|       1|       C001|   120|     VN|
|       2|       C002|    80|     VN|
|       3|       C003|   200|     SG|
|       4|       C001|    50|     VN|
|       5|       C002|    70|     SG|
+--------+-----------+------+-------+



## 2️⃣ GroupBy cơ bản

❓ Yêu cầu

👉 Tổng tiền theo customer_id

In [11]:
df_group = df.groupBy("customer_id").agg(
    F.sum("amount").alias("total_amount")
)

df_group.show()

+-----------+------------+
|customer_id|total_amount|
+-----------+------------+
|       C001|         170|
|       C002|         150|
|       C003|         200|
+-----------+------------+



# 🔥 STOP – Đây là lúc SHUFFLE xảy ra

## 3️⃣ Shuffle là gì? (Hiểu đúng, không mơ hồ)

### 🧠 Định nghĩa CHUẨN:

**Shuffle** = **Spark** phải di chuyển dữ liệu giữa các executor để gom các key giống nhau về cùng 1 nơi

**Ví dụ:**
>*	Order của C001 nằm ở partition 1
>*	Order khác của C001 nằm ở partition 5
>> → Spark bắt buộc phải chuyển dữ liệu

>👉 Network + Disk + Serialize = tốn tài nguyên

### 🧩 Minh họa logic

```code
Partition 1: C001, C002
Partition 2: C003
Partition 3: C001, C002

groupBy(customer_id)
        ↓
Shuffle
        ↓
Partition A: C001
Partition B: C002
Partition C: C003
```



## 4️⃣ Xem Execution Plan (BẮT BUỘC)

Bạn sẽ thấy đoạn giống:

```code
Exchange hashpartitioning(customer_id, 200)
```

📌 Exchange = SHUFFLE

In [12]:
df_group.explain(True)

== Parsed Logical Plan ==
'Aggregate ['customer_id], ['customer_id, 'sum('amount) AS total_amount#64]
+- LogicalRDD [order_id#47L, customer_id#48, amount#49L, country#50], false

== Analyzed Logical Plan ==
customer_id: string, total_amount: bigint
Aggregate [customer_id#48], [customer_id#48, sum(amount#49L) AS total_amount#64L]
+- LogicalRDD [order_id#47L, customer_id#48, amount#49L, country#50], false

== Optimized Logical Plan ==
Aggregate [customer_id#48], [customer_id#48, sum(amount#49L) AS total_amount#64L]
+- Project [customer_id#48, amount#49L]
   +- LogicalRDD [order_id#47L, customer_id#48, amount#49L, country#50], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[customer_id#48], functions=[sum(amount#49L)], output=[customer_id#48, total_amount#64L])
   +- Exchange hashpartitioning(customer_id#48, 4), ENSURE_REQUIREMENTS, [plan_id=119]
      +- HashAggregate(keys=[customer_id#48], functions=[partial_sum(amount#49L)], output=[customer_id#48,

## 5️⃣ Vì sao shuffle NGUY HIỂM?

|Vấn đề|Hậu quả|
|------|-------|
|Nhiều dữ liệu|Chậm|
|Skew key|Executor chết|
|Network yếu|Timeout|
|Disk chậm|Spill|

> 👉 90% job Spark chậm = shuffle kém kiểm soát


## 6️⃣ Ví dụ SHUFFLE TỆ (anti-pattern)

```python
df.groupBy("country", "customer_id").count().show()
```

>❌ GroupBy nhiều cột không cần thiết

>❌ Cardinality cao → shuffle nặng


In [13]:
df.groupBy("country", "customer_id").count().show()

+-------+-----------+-----+
|country|customer_id|count|
+-------+-----------+-----+
|     VN|       C001|    2|
|     VN|       C002|    1|
|     SG|       C002|    1|
|     SG|       C003|    1|
+-------+-----------+-----+



## 7️⃣ Giảm shuffle – cách đầu tiên (cơ bản)

### ✅ Chỉ groupBy đúng thứ cần

```python
df.groupBy("country").sum("amount").show()
```

In [14]:
df.groupBy("country").sum("amount").show()

+-------+-----------+
|country|sum(amount)|
+-------+-----------+
|     VN|        250|
|     SG|        270|
+-------+-----------+



## 8️⃣ Kiểm soát số partition khi shuffle

In [15]:
# Mặc định:

spark.conf.get("spark.sql.shuffle.partitions")

# → thường là 200 (QUÁ NHIỀU cho dataset nhỏ)

'4'

In [9]:
# 🔧 Giảm xuống khi test / small data

spark.conf.set("spark.sql.shuffle.partitions", "4")

# 👉 Chạy lại groupBy và explain

# 🧠 CÂU HỎI & TRẢ LỜI

## 🔹 Spark Lazy Evaluation & Action

### ❓ 1. Vì sao filter() không chạy ngay?

### Trả lời:

**filter()** là **transformation**, **Spark** sử dụng **lazy evaluation**, nên chưa thực thi ngay mà chỉ xây dựng logical execution plan.

**Giải thích ngắn:**
>*	Spark chưa đọc dữ liệu
>*	Chỉ ghi nhớ: “khi nào cần thì filter thế này”

---

### ❓ 2. show() khác collect() ở điểm nào?

### Trả lời:

|**show()**|**collect()**|
|----------|-------------|
|Là action|Là action|
|Chỉ lấy một phần dữ liệu (mặc định 20 dòng)|Lấy toàn bộ dữ liệu|
|An toàn với data lớn|RẤT NGUY HIỂM với data lớn|
|Dùng để debug|Chỉ dùng khi data rất nhỏ|

#### Kết luận:

> ❌ Không dùng collect() trong production

> ✅ Dùng show(), take(), limit()

---

### ❓ 3. Khi nào Spark mới thực sự đọc dữ liệu?

### Trả lời:

> Spark chỉ thực sự đọc và xử lý dữ liệu khi gặp ACTION như:
>> show(), count(), collect(), write()

---

## 🔹 GroupBy & Shuffle:

### ❓ 4. Vì sao groupBy() luôn gây shuffle?

>Vì **Spark** cần gom tất cả các record có cùng **key** về cùng một **executor**, nên bắt buộc phải di chuyển dữ liệu giữa các **partition** → gây **shuffle**.

>> 📌 Không có cách nào groupBy mà không shuffle (trừ vài case rất đặc biệt).

---

###❓ 5. Exchange trong execution plan nghĩa là gì?

### Trả lời:

**Exchange biểu thị giai đoạn shuffle, nơi Spark:**

>*	repartition dữ liệu
>*	truyền dữ liệu qua network
>*	ghi/đọc disk nếu cần

>>📌 Đây là bước đắt nhất trong Spark.

---

###❓ 6. Vì sao spark.sql.shuffle.partitions = 200 nguy hiểm với dataset nhỏ?

### Trả lời:

Vì **Spark** tạo **200 task shuffle**, trong khi dữ liệu rất ít →
**overhead** (task scheduling, network, file) lớn hơn xử lý dữ liệu.

**📌 Với data nhỏ:**
>*	200 partitions = lãng phí
>*	Job chậm hơn thay vì nhanh

---

###❓ 7. Khi nào shuffle bắt buộc, khi nào tránh được?

### Trả lời:

**Shuffle BẮT BUỘC khi:**
>*	groupBy
>*	join (không broadcast)
>*	distinct
>*	orderBy

**Shuffle CÓ THỂ TRÁNH khi:**
>*	filter
>*	select
>*	withColumn
>*	map
>*	limit
>*	broadcast join


# 🚀 PHẦN 2 – Spark Level 2 – Bài 4

## 🔥 repartition() vs coalesce() (RẤT QUAN TRỌNG)

> Đây là chỗ **90% người dùng Spark sai**

---

## 1️⃣ Mục tiêu bài học

**Sau bài này bạn sẽ:**
>* Biết khi nào Spark shuffle
>* Dùng đúng repartition / coalesce
>* Tránh job chậm & crash executor


## 2️⃣ Khởi tạo DataFrame lớn (demo):

Mặc định: ~8 hoặc ~16 partitions (tuỳ Colab)

In [10]:
df = spark.range(0, 1_000_000)
print(df.rdd.getNumPartitions())

2


## 3️⃣ repartition() – LUÔN SHUFFLE

```python
df2 = df.repartition(4)
df2.explain()
```

### 🔥 Điều gì xảy ra?
>*	Spark shuffle toàn bộ dữ liệu
>*	Redistribute đều sang 4 partition

###📌 DÙNG KHI:
>*	Trước join
>*	Trước groupBy
>*	Muốn phân phối lại data đều

> ❌ KHÔNG dùng nếu chỉ muốn giảm partition

In [15]:
df2 = df.repartition(4)
df2.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange RoundRobinPartitioning(4), REPARTITION_BY_NUM, [plan_id=76]
   +- Range (0, 1000000, step=1, splits=2)




## 4️⃣ coalesce() – KHÔNG shuffle (hoặc rất ít)

```python
df3 = df.coalesce(4)
df3.explain()
```

### 🔥 Điều gì xảy ra?
>*	Spark gom partition lại
>*	KHÔNG di chuyển dữ liệu qua network

### 📌 DÙNG KHI:
>*	Sau filter
>*	Trước write
>*	Muốn giảm số file output

In [14]:
df3 = df.coalesce(4)
df3.explain()

== Physical Plan ==
Coalesce 4
+- *(1) Range (0, 1000000, step=1, splits=2)




## 5️⃣ So sánh nhanh (PHẢI NHỚ)

|**Tiêu chí**|**repartition**|**coalesce**|
|------------|---------------|-------------|
|Shuffle|✅ Có|❌ Không|
|Tăng partition|✅|❌|
|Giảm partition|✅|✅|
|Nhanh|❌|✅|
|Dùng trước groupBy|✅|❌|

## 6️⃣ Case thực tế (PRODUCTION)

### ❌ Sai (rất hay gặp)

```python
df.filter("amount > 100").repartition(4)
```
>👉 Shuffle không cần thiết

### ✅ Đúng

```python
df.filter("amount > 100").coalesce(4)
```

### ✅ Trước groupBy

```python
df.repartition("customer_id") \
  .groupBy("customer_id") \
  .sum("amount")
```

> 👉 Shuffle có kiểm soát


## 7️⃣ BÀI TẬP (LÀM TRONG NOTEBOOK)

1.	Tạo DataFrame 1 triệu dòng
2.	So sánh:
*	repartition(4)
*	coalesce(4)
3.	Dùng explain() → tìm Exchange
4.	Ghi nhận xét bằng markdown

> A: ✅ coalesce() không tạo Exchange → không shuffle


In [20]:
# 1.	Tạo DataFrame 1 triệu dòng
df = spark.range(0, 1_000_000)
print(df.rdd.getNumPartitions())

2


In [21]:
# 2.	So sánh:
## repartition
df2 = df.repartition(4)
df2.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange RoundRobinPartitioning(4), REPARTITION_BY_NUM, [plan_id=92]
   +- Range (0, 1000000, step=1, splits=2)




# 🚀 Spark Level 2 – Bài 5: Join Strategies

> ⚠️ **80% job Spark** chậm / **fail OOM** là do **JOIN SAI**

⸻

## 🎯 Mục tiêu bài học

### Sau bài này, bạn sẽ:
>*	Hiểu Spark join hoạt động thế nào
>*	Đọc được physical plan của join
>*	Biết khi nào dùng broadcast join
>*	Tránh shuffle join ngu ngốc
>*	Áp dụng trực tiếp cho Dataproc / BigQuery / CDP

⸻

## 1️⃣ Các loại Join trong Spark

**Spark** KHÔNG chỉ có 1 loại join, mà có nhiều chiến lược thực thi.

### 🔹 4 chiến lược chính:
1.	Broadcast Hash Join ⭐⭐⭐⭐⭐
2.	Shuffle Hash Join
3.	Sort Merge Join
4.	Cartesian / Nested Loop Join (❌ tránh)

⸻

### 2️⃣ Broadcast Join – VUA HIỆU NĂNG

#### 🔥 Khi nào dùng?
>*	1 bảng rất nhỏ
>*	Bảng còn lại rất lớn

#### 📌 Ví dụ thực tế:
>*	orders (100M rows)
>*	customers (10K rows)

#### ✅ Code chuẩn

```python
from pyspark.sql.functions import broadcast

joined = orders.join(
    broadcast(customers),
    "customer_id"
)
```

#### 🔍 Execution plan

```python
joined.explain()
```

Sẽ thấy

```code
BroadcastHashJoin
```

#### 📌 Nghĩa là:
>*	Bảng nhỏ được copy vào RAM của mỗi executor
>*	KHÔNG SHUFFLE bảng lớn
>*	Nhanh nhất có thể

---

#### ❌ Không dùng broadcast khi:
>*	Bảng > vài trăm MB
>*	Executor RAM nhỏ

> 👉 Dễ OOM executor

---

### 3️⃣ Shuffle Join – NGUY HIỂM

#### 🔥 Khi nào xảy ra?
>*	Join 2 bảng lớn
>*	Không có broadcast

```python
orders.join(customers, "customer_id")
```

---

#### 🔍 Execution plan

```code
SortMergeJoin
Exchange
Exchange
```

#### 📌 Ý nghĩa:
>*	Cả 2 bảng đều shuffle
>*	Sort theo join key
>*	Network + disk + CPU = 💸💸💸

---

####❗ Đây là lý do:
>* Job chậm
>*	Disk spill
>*	Executor chết

---

### 4️⃣ Sort Merge Join (mặc định)

Spark dùng Sort Merge Join khi:
>*	Không broadcast được
>*	Data lớn
>*	Key có thể sort

> 📌 Đây là default join strategy

---

### 5️⃣ Shuffle Hash Join (ít gặp)

Xảy ra khi:
>*	Join key nhỏ
>*	spark.sql.join.preferSortMergeJoin = false

> 📌 Hiếm khi dùng trong production.

---

### 6️⃣ Join Strategy Summary (PHẢI NHỚ)

|**Strategy**|**Shuffle**|**Nhanh**|**Khi dùng**|
|------------|-----------|---------|------------|
|BroadcastHashJoin|❌|⭐⭐⭐⭐⭐|1 bảng nhỏ|
|SortMergeJoin|✅|⭐⭐|2 bảng lớn|
|ShuffleHashJoin|✅|⭐⭐|Hiếm|
|Cartesian|❌|💀|Tránh|

---




In [22]:
# Tạo customer dataframe

data = [
    (1, "Nguyễn Văn A", 1987, "IT"),
    (2, "Nguyễn Văn B", 1988, "KD"),
    (3, "Nguyễn Văn C", 1989, "KT"),
    (4, "Nguyễn Văn D", 1990, "IT"),
    (5, "Nguyễn Văn E", 1991, "KD"),
    (6, "Nguyễn Văn F", 1992, "KT"),
    (7, "Nguyễn Văn G", 1993, "IT"),
    (8, "Nguyễn Văn H", 1994, "KD"),
    (9, "Nguyễn Văn I", 1995, "KT"),
    (10, "Nguyễn Văn K", 1996, "IT"),
    (11, "Nguyễn Văn L", 1997, "KD"),
    (12, "Nguyễn Văn M", 1998, "KT"),
    (13, "Nguyễn Văn N", 1999, "IT")
]

cus_df = spark.createDataFrame(data, ["id", "name", "birth_year", "department"])
cus_df.show()

+---+------------+----------+----------+
| id|        name|birth_year|department|
+---+------------+----------+----------+
|  1|Nguyễn Văn A|      1987|        IT|
|  2|Nguyễn Văn B|      1988|        KD|
|  3|Nguyễn Văn C|      1989|        KT|
|  4|Nguyễn Văn D|      1990|        IT|
|  5|Nguyễn Văn E|      1991|        KD|
|  6|Nguyễn Văn F|      1992|        KT|
|  7|Nguyễn Văn G|      1993|        IT|
|  8|Nguyễn Văn H|      1994|        KD|
|  9|Nguyễn Văn I|      1995|        KT|
| 10|Nguyễn Văn K|      1996|        IT|
| 11|Nguyễn Văn L|      1997|        KD|
| 12|Nguyễn Văn M|      1998|        KT|
| 13|Nguyễn Văn N|      1999|        IT|
+---+------------+----------+----------+



In [24]:
# order df

order_data = [
    (1, 1, 100, "VN"),
    (2, 2, 200, "SG"),
    (3, 3, 300, "VN"),
    (4, 4, 400, "SG"),
    (5, 5, 500, "VN"),
    (6, 6, 600, "SG"),
    (7, 7, 700, "VN"),
    (8, 8, 800, "SG"),
    (9, 9, 900, "VN"),
    (10, 10, 1000, "SG"),
    (11, 11, 1100, "VN"),
    (12, 12, 1200, "SG"),
    (13, 13, 1300, "VN")
]

order_df = spark.createDataFrame(order_data, ["id", "customer_id", "amount", "country"])
order_df.show()

+---+-----------+------+-------+
| id|customer_id|amount|country|
+---+-----------+------+-------+
|  1|          1|   100|     VN|
|  2|          2|   200|     SG|
|  3|          3|   300|     VN|
|  4|          4|   400|     SG|
|  5|          5|   500|     VN|
|  6|          6|   600|     SG|
|  7|          7|   700|     VN|
|  8|          8|   800|     SG|
|  9|          9|   900|     VN|
| 10|         10|  1000|     SG|
| 11|         11|  1100|     VN|
| 12|         12|  1200|     SG|
| 13|         13|  1300|     VN|
+---+-----------+------+-------+



### 7️⃣ Bài tập THỰC CHIẾN (RẤT QUAN TRỌNG)

In [28]:
# Setup
from pyspark.sql.functions import col

orders = spark.range(0, 1_000_000) \
    .withColumn("customer_id", (col("id") % 1000))

customers = spark.range(0, 1000) \
    .withColumnRenamed("id", "customer_id")

#### 🔹 Case 1: Join thường (SAI)

👉 Ghi:
>*	Có Exchange không?
>> Có
>*	Strategy gì?
>> SortMergeJoin

#### 🔍 Giải thích
>* 	Bạn KHÔNG dùng broadcast()
>*	Spark sẽ:
>> 1.	Shuffle cả orders
>> 2.	Shuffle cả customers
>> 3.	Sort theo customer_id
>> 4.	Join


In [29]:
orders.join(customers, "customer_id").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#109L, id#108L]
   +- BroadcastHashJoin [customer_id#109L], [customer_id#111L], Inner, BuildRight, false
      :- Project [id#108L, (id#108L % 1000) AS customer_id#109L]
      :  +- Filter isnotnull((id#108L % 1000))
      :     +- Range (0, 1000000, step=1, splits=2)
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=146]
         +- Project [id#110L AS customer_id#111L]
            +- Range (0, 1000, step=1, splits=2)




#### 🔹 Case 2: Broadcast join (ĐÚNG)

👉 Ghi:
>*	Exchange còn không?
>> ❌ Không (phía bảng nhỏ)
>*	Nhanh hơn vì sao?
>> Vì Spark không shuffle bảng lớn,
  chỉ broadcast bảng nhỏ vào RAM của mỗi executor,
  tránh network + disk IO.

#### 🔍 Giải thích

Trong Broadcast Join:
>*	customers (nhỏ):
>*	👉 Broadcast → copy vào RAM executor
>*	❌ Không Exchange
>*	orders (lớn):
>*	❌ Không cần shuffle
>*	Join trực tiếp với hash map trong RAM

In [30]:
from pyspark.sql.functions import broadcast

orders.join(
    broadcast(customers),
    "customer_id"
).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#109L, id#108L]
   +- BroadcastHashJoin [customer_id#109L], [customer_id#111L], Inner, BuildRight, false
      :- Project [id#108L, (id#108L % 1000) AS customer_id#109L]
      :  +- Filter isnotnull((id#108L % 1000))
      :     +- Range (0, 1000000, step=1, splits=2)
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=177]
         +- Project [id#110L AS customer_id#111L]
            +- Range (0, 1000, step=1, splits=2)




#### 🔹 Case 3: Disable broadcast (để học)

```python
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
orders.join(customers, "customer_id").explain()
```

##### ✅ Điều gì xảy ra?

* Spark CẤM broadcast
* Dù bảng nhỏ, Spark vẫn dùng SortMergeJoin
* Exchange xuất hiện ở cả 2 bảng

##### 📌 Execution plan quay lại:

```code
SortMergeJoin
Exchange
Exchange
```

##### 👉 Đây là bằng chứng:
*	Broadcast KHÔNG tự động
*	Spark chỉ broadcast khi:
>*	Bảng nhỏ
>*	Threshold cho phép
>*	Hoặc bạn ép broadcast()

In [31]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

orders.join(customers, "customer_id").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#109L, id#108L]
   +- SortMergeJoin [customer_id#109L], [customer_id#111L], Inner
      :- Sort [customer_id#109L ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(customer_id#109L, 4), ENSURE_REQUIREMENTS, [plan_id=209]
      :     +- Project [id#108L, (id#108L % 1000) AS customer_id#109L]
      :        +- Filter isnotnull((id#108L % 1000))
      :           +- Range (0, 1000000, step=1, splits=2)
      +- Sort [customer_id#111L ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(customer_id#111L, 4), ENSURE_REQUIREMENTS, [plan_id=210]
            +- Project [id#110L AS customer_id#111L]
               +- Range (0, 1000, step=1, splits=2)




## 🧠 TÓM TẮT

### Spark Join Strategy Summary

1. orders.join(customers)
- SortMergeJoin
- Shuffle cả 2 bảng
- Chậm với dữ liệu lớn

2. orders.join(broadcast(customers))
- BroadcastHashJoin
- Không shuffle bảng lớn
- Nhanh nhất

3. Disable autoBroadcastJoinThreshold
- Spark buộc dùng SortMergeJoin
- Dùng để debug / học

# 🚀 Spark Level 2 – Bài 6: Data Skew & Skew Join

## ⚠️ Đây là nguyên nhân số 1 khiến Spark:
*	Chạy mãi không xong
*	1 executor chết
*	Task stuck 99%

---

## 1️⃣ Data Skew là gì?

Data Skew xảy ra khi:
- Một (hoặc vài) key chiếm phần lớn dữ liệu
- Một task phải xử lý quá nhiều records
- Các task khác rảnh → cluster không cân bằng

### 📌 Ví dụ thực tế (ngân hàng):
-	1 merchant cực lớn
-	1 khách hàng VIP
-	1 ngày giao dịch cao điểm

---

## 2️⃣ Dấu hiệu nhận biết Skew
-	Job chạy lâu ở 1 stage
-	Spark UI:
>*	1 task chạy rất lâu
>*	Các task khác DONE
-	Executor OOM / Disk spill

---

## 3️⃣ Ví dụ Skew kinh điển

```python
data = (
    spark.range(0, 10_000_000)
    .withColumn("key",
        when(col("id") < 9_000_000, "HOT")
        .otherwise(col("id"))
    )
)

data.groupBy("key").count().explain()
```

>* 👉 Key "HOT" chiếm 90% dữ liệu
>* 👉 1 reducer gánh hết

---

## 4️⃣ Cách xử lý Skew (CỰC KỲ QUAN TRỌNG)

### 🔹 Cách 1: Salting (thủ công)

```python
from pyspark.sql.functions import rand, concat

salted = data.withColumn(
    "salted_key",
    concat(col("key"), (rand()*10).cast("int"))
)
```

> 👉 Chia 1 key lớn thành nhiều key nhỏ

### 🔹 Cách 2: AQE – Adaptive Query Execution ⭐⭐⭐⭐⭐

```python
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")
```

#### 📌 Spark 3+/4 sẽ:
-	Tự detect skew
-	Tự chia task lớn

> **👉 RẤT KHUYẾN NGHỊ**

### 🔹 Cách 3: Broadcast bảng nhỏ (nếu có)

Skew join + broadcast = 🔥🔥🔥

---

## 5️⃣ Skew Join thực tế trong CDP


|**Case**|**Giải pháp**|
|--------|-------------|
|Merchant lớn|Salting|
|Lookup nhỏ|Broadcast|
|Data Lake lớn|AQE|
|Batch đêm|Repartition trước|

---

## 🧠 CÂU HỎI BẮT BUỘC – BÀI 6

1.	Vì sao skew làm Spark chậm dù cluster còn tài nguyên?
>Data skew làm Spark chậm vì:
>>- Một (hoặc vài) key chiếm phần lớn dữ liệu
>>- Một task phải xử lý lượng dữ liệu vượt trội
>>- Các executor khác rảnh nhưng không thể hỗ trợ
>>- Cluster bị mất cân bằng (resource under-utilization)
> 👉 ĐIỂM CỐT LÕI:
>> Spark không thể chia nhỏ 1 task đang chạy, dù cluster còn CPU/RAM.
2.	Vì sao 1 task chạy lâu làm cả stage bị block?
>Vì một stage chỉ hoàn thành khi TẤT CẢ task trong stage hoàn thành.
>Nếu 1 task bị chậm (straggler), stage bị block,
>kéo theo toàn bộ DAG không thể tiến tiếp.
>> 👉 Đây gọi là Straggler Problem
3.	Salting giải quyết skew bằng cách nào?
> Salting giải quyết skew bằng cách:
>>- Thêm một "salt" ngẫu nhiên vào key
>>- Biến một key lớn thành nhiều key nhỏ
>>- Phân tán dữ liệu sang nhiều task khác nhau
4.	AQE xử lý skew tốt hơn salting ở điểm nào?
>AQE xử lý skew tốt hơn salting vì:
>>- Tự động detect skew dựa trên runtime statistics
>>- Tự chia lại partition lớn thành nhiều partition nhỏ
>>- Không cần thay đổi code
>>- Ít rủi ro logic hơn salting

>**👉 AQE = production best practice (Spark 3+)**
5.	Khi nào KHÔNG nên dùng salting?
>KHÔNG nên dùng salting khi:
>>- Join cần tính chính xác theo key (logic phức tạp)
>>- Dữ liệu đã được AQE xử lý tốt
>>- Lookup table nhỏ có thể broadcast
>>- Pipeline cần đơn giản, dễ bảo trì

**📌 Salting là giải pháp thủ công, dễ gây:**
>*	Bug logic
>*	Code khó maintain

## 🎯 KẾT LUẬN BÀI 6

**👉 Cần HIỂU ĐÚNG:**
>*	Dùng thuật ngữ chính xác hơn
>*	Nhấn mạnh “stage barrier” & “straggler”